In [ ]:
import os
import numpy as np
import glob
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, PReLU, Dense, Flatten, UpSampling2D, Input, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import VGG19

# Check TensorFlow Version
print(f"TensorFlow Version: {tf.__version__}")


In [ ]:
# Define dataset paths
high_res_path = "/kaggle/input/sardatadata/final data/image_chips_native"
low_res_path = "/kaggle/input/sardatadata/final data/low_immage"

# Function to Load Images
def load_images(path, img_size=(416, 416)):
    image_paths = glob.glob(os.path.join(path, "*.*"))  # Accept all image types
    print(f"Found {len(image_paths)} images in {path}")  # Debugging

    images = []
    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Could not read {img_path}")  # Debugging
            continue
        img = cv2.resize(img, img_size)
        img = img / 255.0  # Normalize
        images.append(img)

    return np.array(images, dtype=np.float32)

# Load High-Resolution and Low-Resolution Images
hr_imgs = load_images(high_res_path, img_size=(416, 416))
lr_imgs = load_images(low_res_path, img_size=(208, 208))  # Assuming 2× downsampling

# Check loaded data
print(f"Loaded {len(hr_imgs)} HR images and {len(lr_imgs)} LR images")


In [ ]:
def build_generator():
    inputs = Input(shape=(208, 208, 3))  # Low-resolution image input

    # Initial Convolution Layer
    x = Conv2D(64, kernel_size=9, padding="same")(inputs)
    x = PReLU(shared_axes=[1, 2])(x)

    # Residual Blocks
    def res_block(x):
        res = Conv2D(64, kernel_size=3, padding="same")(x)
        res = BatchNormalization()(res)
        res = PReLU(shared_axes=[1, 2])(res)
        res = Conv2D(64, kernel_size=3, padding="same")(res)
        res = BatchNormalization()(res)
        return Add()([x, res])

    for _ in range(16):  # Use 16 Residual Blocks
        x = res_block(x)

    # Upsampling Layers
    x = UpSampling2D(size=2)(x)
    x = Conv2D(256, kernel_size=3, padding="same")(x)
    x = PReLU(shared_axes=[1, 2])(x)

    x = UpSampling2D(size=2)(x)
    x = Conv2D(256, kernel_size=3, padding="same")(x)
    x = PReLU(shared_axes=[1, 2])(x)

    # Output Layer
    outputs = Conv2D(3, kernel_size=9, padding="same", activation="tanh")(x)

    return Model(inputs, outputs, name="Generator")

generator = build_generator()
generator.summary()


In [ ]:
def build_discriminator():
    inputs = Input(shape=(416, 416, 3))  # High-resolution image input

    def d_block(x, filters, strides=1):
        x = Conv2D(filters, kernel_size=3, strides=strides, padding="same")(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)
        return x

    x = d_block(inputs, 64, strides=1)
    x = d_block(x, 64, strides=2)
    x = d_block(x, 128, strides=1)
    x = d_block(x, 128, strides=2)
    x = d_block(x, 256, strides=1)
    x = d_block(x, 256, strides=2)
    x = d_block(x, 512, strides=1)
    x = d_block(x, 512, strides=2)

    x = Flatten()(x)
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.2)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    return Model(inputs, outputs, name="Discriminator")

discriminator = build_discriminator()
discriminator.summary()


In [ ]:
vgg = VGG19(weights="imagenet", include_top=False, input_shape=(416, 416, 3))
vgg = Model(inputs=vgg.input, outputs=vgg.get_layer("block5_conv4").output)
vgg.trainable = False


In [ ]:
# Compile Models
generator.compile(optimizer=Adam(1e-4), loss="mse")
discriminator.compile(optimizer=Adam(1e-4), loss=BinaryCrossentropy())

batch_size = 4
epochs = 1

# Training Loop
for epoch in range(epochs):
    idx = np.random.randint(0, len(hr_imgs), batch_size)
    lr_batch = lr_imgs[idx]
    hr_batch = hr_imgs[idx]

    # Generate fake HR images
    fake_hr = generator.predict(lr_batch)

    # Train Discriminator
    d_loss_real = discriminator.train_on_batch(hr_batch, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_hr, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator (adversarial + perceptual loss)
    g_loss = generator.train_on_batch(lr_batch, hr_batch)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: D Loss: {d_loss}, G Loss: {g_loss}")

# Save the generator model
generator.save("/kaggle/working/srgan_generator.h5")
print("✅ Model saved successfully!")


In [ ]:
import tensorflow as tf
import numpy as np
from skimage.metrics import structural_similarity as ssim

def calculate_ssim(hr_real, hr_fake):
    # Convert tensors to numpy arrays
    hr_real_np = hr_real.numpy()
    hr_fake_np = hr_fake.numpy()
    
    # Calculate SSIM for each image and take the mean
    ssim_values = [ssim(hr_real_np[i], hr_fake_np[i], multichannel=True) for i in range(hr_real_np.shape[0])]
    return np.mean(ssim_values)

# Example usage inside training loop
ssim_score = calculate_ssim(hr_batch, fake_hr)
print(f'SSIM Score: {ssim_score}')


In [ ]:
# Load the trained model
generator = tf.keras.models.load_model("/kaggle/working/srgan_generator.h5")

# Load a test image and enhance resolution
test_img = cv2.imread("/kaggle/input/sardatadata/final data/low_immage/sample.jpg")  # Change filename
test_img = cv2.resize(test_img, (208, 208))
test_img = np.expand_dims(test_img / 255.0, axis=0)

# Generate High-Resolution Image
sr_img = generator.predict(test_img)
sr_img = np.clip(sr_img[0] * 255, 0, 255).astype("uint8")

# Save Output Image
cv2.imwrite("/kaggle/working/super_resolved.jpg", sr_img)
print("✅ Super-resolved image saved successfully!")
